# лабораторна робота по ИИ

## 1) один нейрон

In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
def neuron(x=[7,4,5], y=0.3, delta_error=0.1):     
    
    if len(x)!=3:
        print('error len of x')
        return 
    if type(y) is not float and type(y) is not int:
        print('error y type is uncorrect')
        return 
    if type(delta_error) is not float and type(delta_error) is not int:
        print('error delta_error type is uncorrect')
        return 
    
    
    # initialization var
    x = np.array([1] + x)
    w = np.random.normal(loc=0.1, scale=0.05, size=(len(x), 1))
    df = pd.DataFrame(columns=['w0', 'w1', 'w2', 'w3', 'y_predict', 'delta'])
    
    def predict(x_list):
        x_list = np.array([1] + x_list)
        s = np.dot(x_list, w)[0]
        y = activization_function(s)
        return y
    
    while True:
        
        s_m = np.dot(x, w)[0]
        y_m = activization_function(s_m)
        
        delta = abs((y_m - y) / y)
        df = df.append({'w0':w[0][0], 'w1':w[1][0], 'w2':w[2][0], 'w3':w[3][0], 'y_predict':y_m, 'delta': delta}, 
                       ignore_index=True)
        
        if delta <= delta_error: 
            return df, predict
        
        eta = y_m*(1-y_m)
        delta_w = eta * (y - y_m) * x
        w = w + delta_w.reshape(len(x), 1)
        

In [12]:
def activization_function(sum_x):
    return 1/(1 + np.exp(-sum_x))

In [4]:
df, prd = neuron()
df[-10:]

,w0,w1,w2,w3,y_predict,delta
0,0.090455,0.131097,0.124785,0.086221,0.874174,1.913915
1,0.027300,-0.310991,-0.127837,-0.229556,0.021694,0.927685
2,0.033207,-0.269644,-0.104210,-0.200023,0.036571,0.878098
3,0.042488,-0.204674,-0.067084,-0.153615,0.081164,0.729455
4,0.058808,-0.090434,-0.001805,-0.072016,0.280592,0.064693


In [5]:
prd([7,4,5])

0.2805921931772059

## 2) Многослойный перцептрон, алгоритм обратного распространения ошибки 1\*1\*1

In [11]:
def Multilayer_perceptron(x=7, y=0.3, delta_error=0.01):
    if type(x) is not float and type(x) is not int:
        print('error x type is uncorrect')
        return 
    if type(y) is not float and type(y) is not int:
        print('error y type is uncorrect')
        return 
    if type(delta_error) is not float and type(delta_error) is not int:
        print('error delta_error type is uncorrect')
        return 
    
    df = pd.DataFrame(columns=['delta', 'y_pr'])
    w12, w23 = np.random.normal(loc=0.1, scale=0.05, size=2)
    
    def predict(x_pr):
        x_s1 = x_pr * w12
        y = activization_function(x_s1)
        
        x_s2 = y * w23
        return activization_function(x_s2)
    
    while True:
        x_sum1 = x * w12
        y2 = activization_function(x_sum1)
        
        x_sum2 = y2 * w23
        y3 = activization_function(x_sum2)
        
        delta = abs((y3 - y) / y)
    
        df = df.append({'delta':delta, 'y_pr':y3}, ignore_index=True)
        
        if delta <= delta_error:
            return df, predict
        
        sigma_3 = y3 * (1-y3) * (y - y3)
        delta_w23 = y2 * sigma_3
        
        sigma_2 = y2 * (1-y2) * (sigma_3 * w23)
        delta_w12 = x * sigma_2
        
        w12 += delta_w12
        w23 += delta_w23
        

In [12]:
df, prd = Multilayer_perceptron()
df[-10:]

,delta,y_pr
86,0.014571,0.304371
87,0.013924,0.304177
88,0.013306,0.303992
89,0.012716,0.303815
90,0.012152,0.303646
91,0.011614,0.303484
92,0.011099,0.303330
93,0.010608,0.303182
94,0.010139,0.303042
95,0.009691,0.302907


In [13]:
prd(6.66)

0.30428897196337795

## 2) многослойный песептрон с одним скрытым слоем

In [31]:
def Multilayer_perceptron_2(x=[2,4,7,3], y=0.3, hidden_layer=40, w1=[], w2=[]):
#     print(pd.DataFrame(w1, columns=['neuron1', 'neuron2', 'neuron3'], index=['x0','x1','x2']))
#     print(pd.DataFrame(w2, index=['y0','neuron1', 'neuron2', 'neuron3'], columns=['exit']))
    x = [1] + list(x)
    y_hidden = np.array([1] + list(activization_function(np.dot(x, w1))))
    y_predict = activization_function(np.dot(y_hidden, w2))
#     print('x\n',x,'\ny_hidden\n',y_hidden, '\ny_predict\n', y_predict)

    delta_exit = y_predict * (1 - y_predict) * (y - y_predict)
    delta_hidden = y_hidden * (1 - y_hidden) * (delta_exit * w2.reshape(hidden_layer + 1))
    
#     print('delta_exit\n',delta_exit,'\delta_hidden\n',delta_hidden)

    
    for i in range(len(x)):
        w1[i,:] += x[i] * delta_hidden[1:]
    
    w2 += y_hidden.reshape(hidden_layer+1, 1) * delta_exit
#     print(pd.DataFrame(w1, columns=['neuron1', 'neuron2', 'neuron3'], index=['x0','x1','x2']))
#     print(pd.DataFrame(w2, index=['y0','neuron1', 'neuron2', 'neuron3'], columns=['exit']))
    return w1, w2
        
        

In [36]:
def online_perceptron(x_list=[[1,2],[2,1],[1,1]], y_list=[0.03, 0.03, 0.02], 
                     count_iter=1000):
    
    hidden_layer = 3
    exit_layer = 1

    w1 = np.random.normal(loc=0.1, scale=0.05, size=(len(x_list[0]) + 1, hidden_layer))
    w2 = np.random.normal(loc=0.1, scale=0.05, size=(hidden_layer + 1, exit_layer))

    def predict(x_pr):
        y_h = np.array([1] + list(activization_function(np.dot([1] + x_pr, w1))))
        return activization_function(np.dot(y_h, w2))
    
    for i in range(count_iter):
        for x, y in zip(x_list, y_list):
            x = list(x)
            w1, w2 = Multilayer_perceptron_2(x=x, y=y, w1=w1, w2=w2, hidden_layer=hidden_layer)
            
    print(predict(x), x , y)
    return predict

In [19]:
x, y = np.arange(0, 5, 0.2), np.arange(0, 5, 0.2)
fun = lambda x,y: x+y
f = [fun(xi,yi) for xi, yi in product(x,y)]


prd = online_perceptron(x_list=list(product(x,y)), y_list=f, count_iter=1000)

     neuron1   neuron2   neuron3
x0  0.082495  0.100586  0.107880
x1  0.057839  0.016665  0.023709
x2  0.119064 -0.025891  0.175187
             exit
y0       0.120660
neuron1  0.133815
neuron2  0.205387
neuron3  0.105184
x
 [1, 0.0, 0.0] 
y_hidden
 [1.         0.52061203 0.5251253  0.52694382] 
y_predict
 [0.58749158]
delta_exit
 [-0.14237578] \delta_hidden
 [-0.         -0.0047549  -0.00729207 -0.00373306]
     neuron1   neuron2   neuron3
x0  0.077740  0.093294  0.104147
x1  0.057839  0.016665  0.023709
x2  0.119064 -0.025891  0.175187
             exit
y0      -0.021716
neuron1  0.059692
neuron2  0.130622
neuron3  0.030160
     neuron1   neuron2   neuron3
x0  0.077740  0.093294  0.104147
x1  0.057839  0.016665  0.023709
x2  0.119064 -0.025891  0.175187
             exit
y0      -0.021716
neuron1  0.059692
neuron2  0.130622
neuron3  0.030160
x
 [1, 0.0, 0.2] 
y_hidden
 [1.         0.5253664  0.52201465 0.53473997] 
y_predict
 [0.52347237]
delta_exit
 [-0.08068988] \delta_hidden
 [ 0.

     neuron1   neuron2   neuron3
x0  0.222261  0.263492  0.237668
x1  0.057839  0.016665  0.023709
x2  0.445350  0.353927  0.478346
             exit
y0       1.483500
neuron1  1.019405
neuron2  1.016200
neuron3  1.027441
     neuron1   neuron2   neuron3
x0  0.222261  0.263492  0.237668
x1  0.057839  0.016665  0.023709
x2  0.445350  0.353927  0.478346
             exit
y0       1.483500
neuron1  1.019405
neuron2  1.016200
neuron3  1.027441
x
 [1, 0.0, 3.6] 
y_hidden
 [1.         0.86122722 0.82311955 0.87650208] 
y_predict
 [0.98367257]
delta_exit
 [0.04202043] \delta_hidden
 [0.         0.00511952 0.00621702 0.00467337]
     neuron1   neuron2   neuron3
x0  0.227380  0.269709  0.242341
x1  0.057839  0.016665  0.023709
x2  0.463780  0.376308  0.495171
             exit
y0       1.525520
neuron1  1.055594
neuron2  1.050787
neuron3  1.064272
     neuron1   neuron2   neuron3
x0  0.227380  0.269709  0.242341
x1  0.057839  0.016665  0.023709
x2  0.463780  0.376308  0.495171
             exit

neuron3  1.235311
     neuron1   neuron2   neuron3
x0  0.240968  0.287768  0.254152
x1  0.057291  0.016186  0.023124
x2  0.543253  0.473538  0.567762
             exit
y0       1.704304
neuron1  1.224554
neuron2  1.214155
neuron3  1.235311
x
 [1, 0.2, 2.0] 
y_hidden
 [1.         0.7923143  0.77522968 0.80127814] 
y_predict
 [0.99010334]
delta_exit
 [0.01185544] \delta_hidden
 [0.         0.00238891 0.00250819 0.00233197]
     neuron1   neuron2   neuron3
x0  0.243357  0.290276  0.256484
x1  0.057769  0.016688  0.023591
x2  0.548031  0.478554  0.572426
             exit
y0       1.716159
neuron1  1.233947
neuron2  1.223346
neuron3  1.244810
     neuron1   neuron2   neuron3
x0  0.243357  0.290276  0.256484
x1  0.057769  0.016688  0.023591
x2  0.548031  0.478554  0.572426
             exit
y0       1.716159
neuron1  1.233947
neuron2  1.223346
neuron3  1.244810
x
 [1, 0.2, 2.2] 
y_hidden
 [1.         0.81162115 0.79354575 0.82061803] 
y_predict
 [0.99107515]
delta_exit
 [0.01246221] \delta_

neuron3  1.391972
     neuron1   neuron2   neuron3
x0  0.262692  0.312075  0.274837
x1  0.061636  0.021048  0.027261
x2  0.608423  0.547327  0.629532
             exit
y0       1.879387
neuron1  1.379972
neuron2  1.366674
neuron3  1.391972
x
 [1, 0.2, 4.800000000000001] 
y_hidden
 [1.         0.96066266 0.94995332 0.96449886] 
y_predict
 [0.99711663]
delta_exit
 [0.01150852] \delta_hidden
 [0.         0.00060016 0.00074776 0.00054852]
     neuron1   neuron2   neuron3
x0  0.263292  0.312823  0.275385
x1  0.061756  0.021197  0.027371
x2  0.611304  0.550917  0.632165
             exit
y0       1.890896
neuron1  1.391028
neuron2  1.377607
neuron3  1.403072
     neuron1   neuron2   neuron3
x0  0.263292  0.312823  0.275385
x1  0.061756  0.021197  0.027371
x2  0.611304  0.550917  0.632165
             exit
y0       1.890896
neuron1  1.391028
neuron2  1.377607
neuron3  1.403072
x
 [1, 0.4, 0.0] 
y_hidden
 [1.         0.57150496 0.57964151 0.57109827] 
y_predict
 [0.9864227]
delta_exit
 [-0.007

x2  0.634320  0.575130  0.654745
             exit
y0       1.937815
neuron1  1.432517
neuron2  1.418278
neuron3  1.444886
x
 [1, 0.4, 3.0] 
y_hidden
 [1.         0.90025567 0.8866025  0.90545973] 
y_predict
 [0.996961]
delta_exit
 [0.00728065] \delta_hidden
 [0.         0.00093653 0.00103816 0.00090051]
     neuron1   neuron2   neuron3
x0  0.272086  0.322313  0.283913
x1  0.065274  0.024993  0.030782
x2  0.637129  0.578245  0.657447
             exit
y0       1.945096
neuron1  1.439072
neuron2  1.424733
neuron3  1.451479
     neuron1   neuron2   neuron3
x0  0.272086  0.322313  0.283913
x1  0.065274  0.024993  0.030782
x2  0.637129  0.578245  0.657447
             exit
y0       1.945096
neuron1  1.439072
neuron2  1.424733
neuron3  1.451479
x
 [1, 0.4, 3.2] 
y_hidden
 [1.         0.91189613 0.89868463 0.91683154] 
y_predict
 [0.99718094]
delta_exit
 [0.00731683] \delta_hidden
 [0.         0.00084595 0.00094916 0.00080981]
     neuron1   neuron2   neuron3
x0  0.272932  0.323263  0.284723

     neuron1   neuron2   neuron3
x0  0.279364  0.330480  0.290903
x1  0.068633  0.028718  0.034024
x2  0.660874  0.605330  0.680094
             exit
y0       2.020483
neuron1  1.509185
neuron2  1.494024
neuron3  1.521882
     neuron1   neuron2   neuron3
x0  0.279364  0.330480  0.290903
x1  0.068633  0.028718  0.034024
x2  0.660874  0.605330  0.680094
             exit
y0       2.020483
neuron1  1.509185
neuron2  1.494024
neuron3  1.521882
x
 [1, 0.6000000000000001, 1.6] 
y_hidden
 [1.         0.79866026 0.7885551  0.80209947] 
y_predict
 [0.99640507]
delta_exit
 [0.00431128] \delta_hidden
 [0.         0.00104626 0.00107397 0.00104151]
     neuron1   neuron2   neuron3
x0  0.280410  0.331553  0.291944
x1  0.069260  0.029362  0.034649
x2  0.662548  0.607048  0.681761
             exit
y0       2.024794
neuron1  1.512628
neuron2  1.497423
neuron3  1.525340
     neuron1   neuron2   neuron3
x0  0.280410  0.331553  0.291944
x1  0.069260  0.029362  0.034649
x2  0.662548  0.607048  0.681761
  

             exit
y0       2.105165
neuron1  1.586463
neuron2  1.570459
neuron3  1.599431
     neuron1   neuron2   neuron3
x0  0.289361  0.341405  0.300640
x1  0.074631  0.035273  0.039867
x2  0.688332  0.635831  0.706689
             exit
y0       2.105165
neuron1  1.586463
neuron2  1.570459
neuron3  1.599431
x
 [1, 0.8, 0.0] 
y_hidden
 [1.         0.58639082 0.59136813 0.58237564] 
y_predict
 [0.99257641]
delta_exit
 [-0.001419] \delta_hidden
 [-0.         -0.00054599 -0.00053851 -0.000552  ]
     neuron1   neuron2   neuron3
x0  0.288815  0.340867  0.300088
x1  0.074194  0.034843  0.039425
x2  0.688332  0.635831  0.706689
             exit
y0       2.103746
neuron1  1.585631
neuron2  1.569620
neuron3  1.598605
     neuron1   neuron2   neuron3
x0  0.288815  0.340867  0.300088
x1  0.074194  0.034843  0.039425
x2  0.688332  0.635831  0.706689
             exit
y0       2.103746
neuron1  1.585631
neuron2  1.569620
neuron3  1.598605
x
 [1, 0.8, 0.2] 
y_hidden
 [1.         0.619125   0.621

KeyboardInterrupt: 

In [24]:
activization_function(0.12+0.133*0.52+0.205*0.525+0.105*0.527)

0.5871315788168427

In [25]:
0.587*(1-0.587)*(0-0.587)

-0.142306997

In [26]:
-0.1423*1+0.12, -0.1423*0.52+0.133, -0.1423*0.525+0.205, -0.1423*0.527+0.105

(-0.022300000000000014, 0.059004, 0.13029249999999998, 0.03000789999999999)

In [ ]:
prd([1,2]), prd([1,1]), prd([1,3])

In [38]:
prd = online_perceptron()
prd([1,1]), prd([2,1]), prd([1,2])

[0.02927067] [1, 1] 0.02


(array([0.02927067]), array([0.02482367]), array([0.02509714]))